In [1]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.getOrCreate()

22/11/05 22:11:01 WARN Utils: Your hostname, kevin resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp0s20f3)
22/11/05 22:11:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/05 22:11:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)

# Key-Value Basics (Key-Value RDDs)
Whenever you see ByKey ina method name, it means that yiu can perform this only on a PairRDD type

In [3]:
words.map(lambda word: (word.lower(), 1))

PythonRDD[1] at RDD at PythonRDD.scala:53

## keyBy

In [4]:
keyword = words.keyBy(lambda word: word.lower()[0])

## mapping over values
If we have a tuple, spark assumes that the first element is the key, and the second is the value. When in this format, you can explicitly choose to map-over the values(and ignore the individual keys). 

In [5]:
keyword.mapValues(lambda word: word.upper()).collect()

[('s', 'SPARK'),
 ('t', 'THE'),
 ('d', 'DEFINITIVE'),
 ('g', 'GUIDE'),
 (':', ':'),
 ('b', 'BIG'),
 ('d', 'DATA'),
 ('p', 'PROCESSING'),
 ('m', 'MADE'),
 ('s', 'SIMPLE')]

In [6]:
keyword.flatMapValues(lambda word: word.upper()).collect()

[('s', 'S'),
 ('s', 'P'),
 ('s', 'A'),
 ('s', 'R'),
 ('s', 'K'),
 ('t', 'T'),
 ('t', 'H'),
 ('t', 'E'),
 ('d', 'D'),
 ('d', 'E'),
 ('d', 'F'),
 ('d', 'I'),
 ('d', 'N'),
 ('d', 'I'),
 ('d', 'T'),
 ('d', 'I'),
 ('d', 'V'),
 ('d', 'E'),
 ('g', 'G'),
 ('g', 'U'),
 ('g', 'I'),
 ('g', 'D'),
 ('g', 'E'),
 (':', ':'),
 ('b', 'B'),
 ('b', 'I'),
 ('b', 'G'),
 ('d', 'D'),
 ('d', 'A'),
 ('d', 'T'),
 ('d', 'A'),
 ('p', 'P'),
 ('p', 'R'),
 ('p', 'O'),
 ('p', 'C'),
 ('p', 'E'),
 ('p', 'S'),
 ('p', 'S'),
 ('p', 'I'),
 ('p', 'N'),
 ('p', 'G'),
 ('m', 'M'),
 ('m', 'A'),
 ('m', 'D'),
 ('m', 'E'),
 ('s', 'S'),
 ('s', 'I'),
 ('s', 'M'),
 ('s', 'P'),
 ('s', 'L'),
 ('s', 'E')]

## Extracting Keys and Values

In [7]:
keyword.keys().collect()

['s', 't', 'd', 'g', ':', 'b', 'd', 'p', 'm', 's']

## lookup

In [8]:
keyword.lookup('s')

['Spark', 'Simple']

# Aggregations
You can perform aggregations on plain RDDs or on PairRDDs, depending on the method that you are using

In [11]:
sc = spark.sparkContext

In [12]:
chars = words.flatMap(lambda word: word.lower())
KVCharacters = chars.map(lambda letter: (letter, 1))

def maxFunc(left, right):
    return max(left, right)

def addFunc(left, right):
    return left + right


nums = sc.parallelize(range(1,31), 5)

## countByKey
You can count the number of elements for each key, collecting the results to a local map. You can also do this with an approximation

In [13]:
KVCharacters.countByKey()

defaultdict(int,
            {'s': 4,
             'p': 3,
             'a': 4,
             'r': 2,
             'k': 1,
             't': 3,
             'h': 1,
             'e': 7,
             'd': 4,
             'f': 1,
             'i': 7,
             'n': 2,
             'v': 1,
             'g': 3,
             'u': 1,
             ':': 1,
             'b': 1,
             'o': 1,
             'c': 1,
             'm': 2,
             'l': 1})

## Understanding aggregation implementations
There are several ways to create your key-value pairRDDS; however, the implementation is important for job stability. Let us compare two fundamental choices, groupBy and reduce. 

### groupByKey
This is a map over eacyh grouping which is the best way to sum up the counts for each key

In [14]:
from functools import reduce
KVCharacters.groupByKey().map(lambda row: (row[0], reduce(addFunc, row[1]))).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

In [15]:
spark.stop()